In [ ]:
# @title Modules 
# Mounting Drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import drive

# Keras modules
import keras
from keras.models import Sequential
from keras.layers import Dense, Convolution2D, Flatten, MaxPooling2D, ZeroPadding2D, Dropout, BatchNormalization, Activation, DepthwiseConv2D, SeparableConv2D
from keras import optimizers
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from keras.callbacks import ModelCheckpoint, CSVLogger, RemoteMonitor, TensorBoard, ReduceLROnPlateau, History, EarlyStopping
from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback
import keras.backend as K
from tqdm.keras import TqdmCallback

#General
import numpy as np
from scipy import signal
from scipy.io import wavfile
import matplotlib.pyplot as plt
import glob
import datetime
import os
import pickle
from scipy.stats import mode
import tarfile
from tqdm import tqdm
import datetime
import math

#auth.authenticate_user()
#gauth = GoogleAuth()
#gauth.credentials = GoogleCredentials.get_application_default()
#drive = GoogleDrive(gauth)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# @title Import our modules
%cd /content/drive/MyDrive/EPFL Account/ML Project 2/ml_project2/src
# specify the location .../ml_project2/src
#Our Modules
import sound_processing as sp
from utils import*

/content/drive/MyDrive/EPFL Account/ML Project 2/ml_project2/src


The best models are already predefined in the code.

In [ ]:
# @title Predicts the patients outcome for POA2 with  the model trained POA1. 
def predict_each_patient():
  """Predicts the patients outcome for POA2 with  the model trained POA1. To get the
   same data as in PDF need to rerun this for batch3 and batch4 """
  (features, positions, controls, patientnbs), (featuresT, positionsT, controlsT, patientnbsT) = get_POA(batch=2)
  x, y, patient_id = stack_pos(_, features, controls, positions, patientnbs)
  #x_test, y_test, patient_id_test = stack_pos(_, featuresT, controlsT, positionsT, patientnbsT)
  #x = np.row_stack([x_train, x_test])
  #y = np.row_stack([y_train, y_test])
  #patient_id = np.hstack([patient_id_train, patient_id_test])

  model = keras.models.load_model('./models/TRAIN_ON_POA_BATCH_1/TRAIN_ON_POA_BATCH_1_RUN_7_-2020-12-12.h5', custom_objects={'f1': f1})
  #the best model for this combo
  y_prob = model.predict(x)

  y_exp = []
  y_mode = []
  y_max = []
  y_mean = []
  for patient in np.unique(patient_id):
    indx = patient_id == patient
    y_exp.append(np.argmax(y[indx][0]))

    y_mode.append(mode(np.argmax(y_prob[indx], axis=1))[0][0])
    y_mean.append(np.argmax(np.mean(y_prob[indx], axis=0)))
    y_max.append(np.argmax(np.max(y_prob[indx], axis=0)))

  y_mode = np.array(y_mode)
  y_mean = np.array(y_mean)
  y_max = np.array(y_max)

  print('There are ' + str(np.unique(patient_id).shape[0]) + ' patients\n')

  print('\nWith mean method')
  print('Accuracy: ' + str(round(np.mean(y_mean == y_exp)*100, 2)) + '%')
  print('Confusion Matrix')
  confusion(y_mean, y_exp)


  print('\nWith max method')
  print('Accuracy: ' + str(round(np.mean(y_max == y_exp)*100, 2)) + '%')
  print('Confusion Matrix')
  confusion(y_max, y_exp)

  print('\nWith mode method')
  print('Accuracy: ' + str(round(np.mean(y_mode == y_exp)*100, 2)) + '%')
  print('Confusion Matrix')
  confusion(y_mode, y_exp)
  

predict_each_patient()

Importing from: ./npz_files/POA/POA_Ca_train_b2.npz
Importing from: ./npz_files/POA/POA_Co_train_b2.npz
Importing from: ./npz_files/POA/POA_Ca_Co_test.npz
There are 66 patients


With mean method
Accuracy: 86.36%
Confusion Matrix
TP: 34, TN: 23, FP: 3, FN: 6

With max method
Accuracy: 84.85%
Confusion Matrix
TP: 34, TN: 22, FP: 4, FN: 6

With mode method
Accuracy: 87.88%
Confusion Matrix
TP: 34, TN: 24, FP: 2, FN: 6


In [ ]:
# @title Predicts the patients outcome for POA1 with  the model trained POA2.
   
def predict_each_patient():
  """Predicts the patients outcome for POA1 with  the model trained POA2. To get the
   same data as in PDF need to rerun this for batch3 and batch4 """
  (features, positions, controls, patientnbs), (featuresT, positionsT, controlsT, patientnbsT) = get_POA(batch=1)
  x, y, patient_id = stack_pos(_, features, controls, positions, patientnbs)
  #x_test, y_test, patient_id_test = stack_pos(_, featuresT, controlsT, positionsT, patientnbsT)
  #x = np.row_stack([x_train, x_test])
  #y = np.row_stack([y_train, y_test])
  #patient_id = np.hstack([patient_id_train, patient_id_test])

  model = keras.models.load_model('./models/TRAIN_ON_POA_BATCH_2/TRAIN_ON_POA_BATCH_2_RUN_9_-2020-12-12.h5', custom_objects={'f1': f1})
  y_prob = model.predict(x)

  y_exp = []
  y_mode = []
  y_max = []
  y_mean = []
  for patient in np.unique(patient_id):
    indx = patient_id == patient
    y_exp.append(np.argmax(y[indx][0]))

    y_mode.append(mode(np.argmax(y_prob[indx], axis=1))[0][0])
    y_mean.append(np.argmax(np.mean(y_prob[indx], axis=0)))
    y_max.append(np.argmax(np.max(y_prob[indx], axis=0)))

  y_mode = np.array(y_mode)
  y_mean = np.array(y_mean)
  y_max = np.array(y_max)

  print('There are ' + str(np.unique(patient_id).shape[0]) + ' patients\n')

  print('\nWith mean method')
  print('Accuracy: ' + str(round(np.mean(y_mean == y_exp)*100, 2)) + '%')
  print('Confusion Matrix')
  confusion(y_mean, y_exp)


  print('\nWith max method')
  print('Accuracy: ' + str(round(np.mean(y_max == y_exp)*100, 2)) + '%')
  print('Confusion Matrix')
  confusion(y_max, y_exp)

  print('\nWith mode method')
  print('Accuracy: ' + str(round(np.mean(y_mode == y_exp)*100, 2)) + '%')
  print('Confusion Matrix')
  confusion(y_mode, y_exp)
  

predict_each_patient()

Importing from: ./npz_files/POA/POA_Ca_train_b1.npz
Importing from: ./npz_files/POA/POA_Co_train_b1.npz
Importing from: ./npz_files/POA/POA_Ca_Co_test.npz
There are 90 patients


With mean method
Accuracy: 65.56%
Confusion Matrix
TP: 38, TN: 21, FP: 29, FN: 2

With max method
Accuracy: 71.11%
Confusion Matrix
TP: 39, TN: 25, FP: 25, FN: 1

With mode method
Accuracy: 65.56%
Confusion Matrix
TP: 38, TN: 21, FP: 29, FN: 2


In [ ]:
# @title Predicts the patients outcome for POA2 with  the model trained GVA and transfered onto POA1
def predict_each_patient():
  """Predicts the patients outcome for POA2 with  the model trained GVA and transfered onto POA1. To get the
   same data as in PDF need to rerun this for batch3 and batch4 """
  (features, positions, controls, patientnbs), (featuresT, positionsT, controlsT, patientnbsT) = get_POA(batch=2)
  x, y, patient_id = stack_pos(_, features, controls, positions, patientnbs)
  #x_test, y_test, patient_id_test = stack_pos(_, featuresT, controlsT, positionsT, patientnbsT)
  #x = np.row_stack([x_train, x_test])
  #y = np.row_stack([y_train, y_test])
  #patient_id = np.hstack([patient_id_train, patient_id_test])

  model = keras.models.load_model('./models/TRAIN_ON_GVA_TRANSF_POA_BATCH_1/TRAIN_ON_GVA_TRANSF_POA_BATCH_1_RUN_2_-2020-12-12.h5', custom_objects={'f1': f1})
  y_prob = model.predict(x)

  y_exp = []
  y_mode = []
  y_max = []
  y_mean = []
  for patient in np.unique(patient_id):
    indx = patient_id == patient
    y_exp.append(np.argmax(y[indx][0]))

    y_mode.append(mode(np.argmax(y_prob[indx], axis=1))[0][0])
    y_mean.append(np.argmax(np.mean(y_prob[indx], axis=0)))
    y_max.append(np.argmax(np.max(y_prob[indx], axis=0)))

  y_mode = np.array(y_mode)
  y_mean = np.array(y_mean)
  y_max = np.array(y_max)

  print('There are ' + str(np.unique(patient_id).shape[0]) + ' patients\n')

  print('\nWith mean method')
  print('Accuracy: ' + str(round(np.mean(y_mean == y_exp)*100, 2)) + '%')
  print('Confusion Matrix')
  confusion(y_mean, y_exp)


  print('\nWith max method')
  print('Accuracy: ' + str(round(np.mean(y_max == y_exp)*100, 2)) + '%')
  print('Confusion Matrix')
  confusion(y_max, y_exp)

  print('\nWith mode method')
  print('Accuracy: ' + str(round(np.mean(y_mode == y_exp)*100, 2)) + '%')
  print('Confusion Matrix')
  confusion(y_mode, y_exp)
  

predict_each_patient()

Importing from: ./npz_files/POA/POA_Ca_train_b2.npz
Importing from: ./npz_files/POA/POA_Co_train_b2.npz
Importing from: ./npz_files/POA/POA_Ca_Co_test.npz
There are 66 patients


With mean method
Accuracy: 95.45%
Confusion Matrix
TP: 39, TN: 24, FP: 2, FN: 1

With max method
Accuracy: 92.42%
Confusion Matrix
TP: 39, TN: 22, FP: 4, FN: 1

With mode method
Accuracy: 95.45%
Confusion Matrix
TP: 39, TN: 24, FP: 2, FN: 1


In [ ]:
# @title Predicts the patients outcome for POA1 with  the model trained GVA and transfered onto POA2
def predict_each_patient():
  """Predicts the patients outcome for POA1 with  the model trained GVA and transfered onto POA2. To get the
   same data as in PDF need to rerun this for batch3 and batch4 """
  (features, positions, controls, patientnbs), (featuresT, positionsT, controlsT, patientnbsT) = get_POA(batch=1)
  x, y, patient_id = stack_pos(_, features, controls, positions, patientnbs)
  #x_test, y_test, patient_id_test = stack_pos(_, featuresT, controlsT, positionsT, patientnbsT)
  #x = np.row_stack([x_train, x_test])
  #y = np.row_stack([y_train, y_test])
  #patient_id = np.hstack([patient_id_train, patient_id_test])

  model = keras.models.load_model('./models/TRAIN_ON_GVA_TRANSF_POA_BATCH_2/TRAIN_ON_GVA_TRANSF_POA_BATCH_2_RUN_6_-2020-12-12.h5', custom_objects={'f1': f1})
  y_prob = model.predict(x)

  y_exp = []
  y_mode = []
  y_max = []
  y_mean = []
  for patient in np.unique(patient_id):
    indx = patient_id == patient
    #y_prob = np.argmax(y_prob)
    #pred = mode(y_prob[indx], axis=0)
    #print('predicted {}, expected {}'.format(np.argmax(prob), np.argmax(y_test[indx][0])))
    y_exp.append(np.argmax(y[indx][0]))

    y_mode.append(mode(np.argmax(y_prob[indx], axis=1))[0][0])
    y_mean.append(np.argmax(np.mean(y_prob[indx], axis=0)))
    y_max.append(np.argmax(np.max(y_prob[indx], axis=0)))

  y_mode = np.array(y_mode)
  y_mean = np.array(y_mean)
  y_max = np.array(y_max)

  print('There are ' + str(np.unique(patient_id).shape[0]) + ' patients\n')

  print('\nWith mean method')
  print('Accuracy: ' + str(round(np.mean(y_mean == y_exp)*100, 2)) + '%')
  print('Confusion Matrix')
  confusion(y_mean, y_exp)


  print('\nWith max method')
  print('Accuracy: ' + str(round(np.mean(y_max == y_exp)*100, 2)) + '%')
  print('Confusion Matrix')
  confusion(y_max, y_exp)

  print('\nWith mode method')
  print('Accuracy: ' + str(round(np.mean(y_mode == y_exp)*100, 2)) + '%')
  print('Confusion Matrix')
  confusion(y_mode, y_exp)
  

predict_each_patient()

Importing from: ./npz_files/POA/POA_Ca_train_b1.npz
Importing from: ./npz_files/POA/POA_Co_train_b1.npz
Importing from: ./npz_files/POA/POA_Ca_Co_test.npz
There are 90 patients


With mean method
Accuracy: 71.11%
Confusion Matrix
TP: 37, TN: 27, FP: 23, FN: 3

With max method
Accuracy: 71.11%
Confusion Matrix
TP: 35, TN: 29, FP: 21, FN: 5

With mode method
Accuracy: 71.11%
Confusion Matrix
TP: 37, TN: 27, FP: 23, FN: 3


In [ ]:
# @title Predicts the patients outcome for GVA with  the model trained on POA1 transfered onto GVA
def predict_each_patient():
  """Predicts the patients outcome for GVA with  the model trained on POA1 transfered onto GVA. """
  (features, positions, controls, patientnbs), (featuresT, positionsT, controlsT, patientnbsT) = get_GVA()
  x_train, y_train, patient_id_train = stack_pos(_, features, controls, positions, patientnbs)
  x_test, y_test, patient_id_test = stack_pos(_, featuresT, controlsT, positionsT, patientnbsT)
  x = np.row_stack([x_train, x_test])
  y = np.row_stack([y_train, y_test])
  patient_id = np.hstack([patient_id_train, patient_id_test])


  model = keras.models.load_model('./models/TRAIN_ON_POA_BATCH_1_TRANSF_GVA/TRAIN_ON_POA_BATCH_1_TRANSF_GVA_RUN_5_-2020-12-12.h5', custom_objects={'f1': f1})
  y_prob = model.predict(x)

  y_exp = []
  y_mode = []
  y_max = []
  y_mean = []
  for patient in np.unique(patient_id):
    indx = patient_id == patient
    #y_prob = np.argmax(y_prob)
    #pred = mode(y_prob[indx], axis=0)
    #print('predicted {}, expected {}'.format(np.argmax(prob), np.argmax(y_test[indx][0])))
    y_exp.append(np.argmax(y[indx][0]))

    y_mode.append(mode(np.argmax(y_prob[indx], axis=1))[0][0])
    y_mean.append(np.argmax(np.mean(y_prob[indx], axis=0)))
    y_max.append(np.argmax(np.max(y_prob[indx], axis=0)))

  y_mode = np.array(y_mode)
  y_mean = np.array(y_mean)
  y_max = np.array(y_max)

  print('There are ' + str(np.unique(patient_id).shape[0]) + ' patients\n')

  print('\nWith mean method')
  print('Accuracy: ' + str(round(np.mean(y_mean == y_exp)*100, 2)) + '%')
  print('Confusion Matrix')
  confusion(y_mean, y_exp)


  print('\nWith max method')
  print('Accuracy: ' + str(round(np.mean(y_max == y_exp)*100, 2)) + '%')
  print('Confusion Matrix')
  confusion(y_max, y_exp)

  print('\nWith mode method')
  print('Accuracy: ' + str(round(np.mean(y_mode == y_exp)*100, 2)) + '%')
  print('Confusion Matrix')
  confusion(y_mode, y_exp)
  

predict_each_patient()

Importing from: ./npz_files/GVA/GVA_Ca_train_b1.npz
Importing from: ./npz_files/GVA/GVA_Co_train_b1.npz
Importing from: ./npz_files/GVA/GVA_Ca_Co_test.npz
There are 78 patients


With mean method
Accuracy: 93.59%
Confusion Matrix
TP: 52, TN: 21, FP: 2, FN: 3

With max method
Accuracy: 92.31%
Confusion Matrix
TP: 51, TN: 21, FP: 2, FN: 4

With mode method
Accuracy: 93.59%
Confusion Matrix
TP: 52, TN: 21, FP: 2, FN: 3


In [ ]:
# @title Predicts the patients outcome for GVA with  the model trained  on on trained on POA2 transfered onto GVA
def predict_each_patient():
  """Predicts the patients outcome for GVA with  the model trained  on trained on POA2 transfered onto GVA. """
  (features, positions, controls, patientnbs), (featuresT, positionsT, controlsT, patientnbsT) = get_GVA()
  x_train, y_train, patient_id_train = stack_pos(_, features, controls, positions, patientnbs)
  x_test, y_test, patient_id_test = stack_pos(_, featuresT, controlsT, positionsT, patientnbsT)
  x = np.row_stack([x_train, x_test])
  y = np.row_stack([y_train, y_test])
  patient_id = np.hstack([patient_id_train, patient_id_test])


  model = keras.models.load_model('./models/TRAIN_ON_POA_BATCH_2_TRANSF_GVA/TRAIN_ON_POA_BATCH_2_TRANSF_GVA_RUN_8_-2020-12-13.h5', custom_objects={'f1': f1})
  y_prob = model.predict(x)

  y_exp = []
  y_mode = []
  y_max = []
  y_mean = []
  for patient in np.unique(patient_id):
    indx = patient_id == patient
    #y_prob = np.argmax(y_prob)
    #pred = mode(y_prob[indx], axis=0)
    #print('predicted {}, expected {}'.format(np.argmax(prob), np.argmax(y_test[indx][0])))
    y_exp.append(np.argmax(y[indx][0]))

    y_mode.append(mode(np.argmax(y_prob[indx], axis=1))[0][0])
    y_mean.append(np.argmax(np.mean(y_prob[indx], axis=0)))
    y_max.append(np.argmax(np.max(y_prob[indx], axis=0)))

  y_mode = np.array(y_mode)
  y_mean = np.array(y_mean)
  y_max = np.array(y_max)

  print('There are ' + str(np.unique(patient_id).shape[0]) + ' patients\n')

  print('\nWith mean method')
  print('Accuracy: ' + str(round(np.mean(y_mean == y_exp)*100, 2)) + '%')
  print('Confusion Matrix')
  confusion(y_mean, y_exp)


  print('\nWith max method')
  print('Accuracy: ' + str(round(np.mean(y_max == y_exp)*100, 2)) + '%')
  print('Confusion Matrix')
  confusion(y_max, y_exp)

  print('\nWith mode method')
  print('Accuracy: ' + str(round(np.mean(y_mode == y_exp)*100, 2)) + '%')
  print('Confusion Matrix')
  confusion(y_mode, y_exp)
  

predict_each_patient()

Importing from: ./npz_files/GVA/GVA_Ca_train_b1.npz
Importing from: ./npz_files/GVA/GVA_Co_train_b1.npz
Importing from: ./npz_files/GVA/GVA_Ca_Co_test.npz
There are 78 patients


With mean method
Accuracy: 78.21%
Confusion Matrix
TP: 39, TN: 22, FP: 1, FN: 16

With max method
Accuracy: 74.36%
Confusion Matrix
TP: 36, TN: 22, FP: 1, FN: 19

With mode method
Accuracy: 79.49%
Confusion Matrix
TP: 40, TN: 22, FP: 1, FN: 15


In [ ]:
# @title Predicts the patients outcome for GVA with the model that was trained on GVA
def predict_each_patient():
  """Predicts the patients outcome for GVA with the model that was trained on GVA"""
  (features, positions, controls, patientnbs), (featuresT, positionsT, controlsT, patientnbsT) = get_GVA()
  x_train, y_train, patient_id_train = stack_pos(_, features, controls, positions, patientnbs)
  x_test, y_test, patient_id_test = stack_pos(_, featuresT, controlsT, positionsT, patientnbsT)
  x = np.row_stack([x_train, x_test])
  y = np.row_stack([y_train, y_test])
  patient_id = np.hstack([patient_id_train, patient_id_test])

  model = keras.models.load_model('./models/TRAIN_ON_GVA_REM_POS_9/TRAIN_ON_GVA_TFINAL_RUN_3_-2020-12-12.h5', custom_objects={'f1': f1})
  y_prob = model.predict(x)

  y_exp = []
  y_mode = []
  y_max = []
  y_mean = []
  for patient in np.unique(patient_id):
    indx = patient_id == patient
    y_exp.append(np.argmax(y[indx][0]))

    y_mode.append(mode(np.argmax(y_prob[indx], axis=1))[0][0])
    y_mean.append(np.argmax(np.mean(y_prob[indx], axis=0)))
    y_max.append(np.argmax(np.max(y_prob[indx], axis=0)))

  y_mode = np.array(y_mode)
  y_mean = np.array(y_mean)
  y_max = np.array(y_max)

  print('There are ' + str(np.unique(patient_id).shape[0]) + ' patients\n')

  print('\nWith mean method')
  print('Accuracy: ' + str(round(np.mean(y_mean == y_exp)*100, 2)) + '%')
  print('Confusion Matrix')
  confusion(y_mean, y_exp)


  print('\nWith max method')
  print('Accuracy: ' + str(round(np.mean(y_max == y_exp)*100, 2)) + '%')
  print('Confusion Matrix')
  confusion(y_max, y_exp)

  print('\nWith mode method')
  print('Accuracy: ' + str(round(np.mean(y_mode == y_exp)*100, 2)) + '%')
  print('Confusion Matrix')
  confusion(y_mode, y_exp)
  

predict_each_patient()

Importing from: ./npz_files/GVA/GVA_Ca_train_b1.npz
Importing from: ./npz_files/GVA/GVA_Co_train_b1.npz
Importing from: ./npz_files/GVA/GVA_Ca_Co_test.npz
There are 78 patients


With mean method
Accuracy: 100.0%
Confusion Matrix
TP: 55, TN: 23, FP: 0, FN: 0

With max method
Accuracy: 100.0%
Confusion Matrix
TP: 55, TN: 23, FP: 0, FN: 0

With mode method
Accuracy: 97.44%
Confusion Matrix
TP: 54, TN: 22, FP: 1, FN: 1


In [ ]:
def print_stat():
  """Accesses the log files written during training and prints the aggregate
   statistics of the model as well as the number of the best position """
  
  full_path  = 'models/TRAIN_ON_GVA_REM_POS_9' #specify the location of the folder
  #with logs


  val_loss = []
  val_acc = []  
  val_f1 = []
  for file in os.listdir(full_path):
    if file.endswith(".txt"):
      acc, f1, loss = np.loadtxt(os.path.join(full_path, file), delimiter=',', skiprows=1, usecols=(4, 5, 6), unpack=True)
      val_loss.append(np.min(loss))
      val_acc.append(acc[np.argmin(loss)])
      val_f1.append(f1[np.argmin(loss)])
  val_loss = np.array(val_loss)
  val_acc = np.array(val_acc)
  val_f1 = np.array(val_f1)


  print('Acc: Mean {}, STD: {}'.format(np.mean(val_acc), np.std(val_acc)))   
  print('F1: Mean {}, STD: {}'.format(np.mean(val_f1), np.std(val_f1)))   
  print('Loss: Mean {}, STD: {}'.format(np.mean(val_loss), np.std(val_loss)))   

  print('Best Model by loss {}, Loss: {}, Acc: {}, F1: {}'.format(np.argmin(val_loss) + 1, np.min(val_loss), val_acc[np.argmin(val_loss)], val_f1[np.argmin(val_loss)]))
  print('Best Model by Acc {}, Loss: {}, Acc: {}, F1: {}'.format(np.argmax(val_acc) + 1, val_loss[np.argmax(val_acc)], np.max(val_acc), val_acc[np.argmax(val_acc)] ))
  print('Best Model by F1 {}, Loss: {}, Acc: {}, F1: {}'.format(np.argmax(val_f1) + 1, val_loss[np.argmax(val_f1)], val_acc[np.argmax(val_f1)], np.max(val_f1)))

print_stat()

Acc: Mean 0.882142859697342, STD: 0.0563284750002707
F1: Mean 0.8687499403953552, STD: 0.06953977229954862
Loss: Mean 0.318777097761631, STD: 0.09177959060773921
Best Model by loss 3, Loss: 0.14865784347057343, Acc: 0.9795918464660645, F1: 0.9508928656578064
Best Model by Acc 3, Loss: 0.14865784347057343, Acc: 0.9795918464660645, F1: 0.9795918464660645
Best Model by F1 3, Loss: 0.14865784347057343, Acc: 0.9795918464660645, F1: 0.9508928656578064
